### [Example 1: Scalar SDEs](https://docs.sciml.ai/stable/tutorials/sde_example/#Stochastic-Differential-Equations-1)

In [ ]:
using DifferentialEquations
using Plots

In [ ]:
# import Pkg
# Pkg.add("ORCA")

In [ ]:
α=1
β=1
u0=1/2
f(u,p,t) = α*u
g(u,p,t) = β*u
dt = 1//2^(4)
tspan = (0.0,1.0)
prob = SDEProblem(f,g,u0,(0.0,1.0))

In [ ]:
# classic Euler-Maruyama algorithm EM
sol = solve(prob,EM(),dt=dt)
plotly() # Using the Plotly backend
plot(sol)

### Using Higher Order Methods

In [ ]:
f_analytic(u0,p,t,W) = u0 * exp((α - (β^2) / 2) * t + β * W)
ff = SDEFunction(f, g, analytic=f_analytic)
prob = SDEProblem(ff, g, u0, (0.0,1.0))

In [ ]:
#We can plot using the classic Euler-Maruyama algorithm as follows:
sol = solve(prob,EM(),dt=dt)
plot(sol,plot_analytic=true)

In [ ]:
# We can choose a higher-order solver for a more accurate result:
sol = solve(prob, SRIW1(),dt=dt,adaptive=false)
plot(sol,plot_analytic=true)

By default, the higher order methods have adaptivity. Thus one can use

In [ ]:
sol = solve(prob,SRIW1())
plot(sol,plot_analytic=true)

Here we allowed the solver to automatically determine a starting dt. This estimate at the beginning is conservative (small) to ensure accuracy. We can instead start the method with a larger dt by passing in a value for the starting dt:

In [ ]:
sol = solve(prob,SRIW1(),dt=dt)
plot(sol,plot_analytic=true)

### Ensemble Simulations

In [ ]:
ensembleprob = EnsembleProblem(prob)

In [ ]:
sol = solve(ensembleprob, EnsembleThreads(), trajectories=1000)

In [ ]:
using Base.Threads
nthreads()

In [ ]:
using DifferentialEquations.EnsembleAnalysis
summ = EnsembleSummary(sol,0:0.01:1)
plot(summ,labels="Middle 95%")
summ = EnsembleSummary(sol,0:0.01:1;quantiles=[0.25,0.75])
plot!(summ,labels="Middle 50%",legend=true)

### Example 2: Systems of SDEs with Diagonal Noise

For example, the Lorenz equation with additive noise has the same deterministic portion as the Lorenz equations, but adds an additive noise, which is simply `3*N(0,dt)` where `N` is the normal distribution `dt` is the time step, to each step of the equation. This is done via:

In [ ]:
function lorenz(du,u,p,t)
  du[1] = 10.0 * (u[2] - u[1])
  du[2] = u[1] * (28.0 - u[3]) - u[2]
  du[3] = u[1] * u[2] - (8/3) * u[3]
end

function σ_lorenz(du,u,p,t)
  du[1] = 3.0
  du[2] = 3.0
  du[3] = 3.0
end


prob_sde_lorenz = SDEProblem(
    lorenz,
    σ_lorenz,
    [1.0,0.0,0.0],
    (0.0, 100.0))
sol = solve(prob_sde_lorenz)
plot(sol,vars=(1,2,3))

In [ ]:
# passing parameters
function lorenz(du,u,p,t)
  du[1] = p[1] * (u[2] - u[1])
  du[2] = u[1] * (p[2] - u[3]) - u[2]
  du[3] = u[1] * u[2] - p[3] * u[3]
end

function σ_lorenz(du,u,p,t)
  du[1] = 3.0
  du[2] = 3.0
  du[3] = 3.0
end

p = [10.0, 28.0, 8/3]
prob_sde_lorenz = SDEProblem(
    lorenz,
    σ_lorenz,
    [1.0,0.0,0.0],
    (0.0, 20.0), p)
sol = solve(prob_sde_lorenz)
plot(sol,vars=(1,2,3))

Note that it's okay for the noise function to mix terms. For example

In [ ]:
function lorenz(du,u,p,t)
  du[1] = p[1] * (u[2] - u[1])
  du[2] = u[1] * (p[2] - u[3]) - u[2]
  du[3] = u[1] * u[2] - p[3] * u[3]
end

function σ_lorenz(du,u,p,t)
  du[1] = sin(u[3])*3.0
  du[2] = u[2]*u[1]*3.0
  du[3] = 3.0
end

p = [10.0, 28.0, 8/3]
prob_sde_lorenz = SDEProblem(
    lorenz,
    σ_lorenz,
    [1.0,0.0,0.0],
    (0.0, 20.0), p)
sol = solve(prob_sde_lorenz)
plot(sol,vars=(1,2,3))

### Example 3: Systems of SDEs with Scalar Noise

In [ ]:
f(du,u,p,t) = (du .= u)
g(du,u,p,t) = (du .= u)
u0 = rand(4, 2)

W = WienerProcess(0.0,0.0,0.0)
prob = SDEProblem(f,g,u0,(0.0,1.0),noise=W)
sol = solve(prob,SRIW1())
plot(sol)